# Chapter 5. Data Class Builders
"Data classes are like children. They are okay as a starting point, but to participate as a grownup object, they need to take some responsibility."

Python offers a few ways to build a simple class that is just a collection of fields, with little or no extra functionality. That pattern is known as a “data class”—and dataclasses is one of the packages that supports this pattern.

This chapter covers three different class builders that you may use as shortcuts to write data classes:

`collections.namedtuple`
    - The simplest way—available since Python 2.6.

`typing.NamedTuple`
    - An alternative that requires type hints on the fields—since Python 3.5, with class syntax added in 3.6.

`@dataclasses.dataclass`
    - A class decorator that allows more customization than previous alternatives, adding lots of options and potential complexity—since Python 3.7.


`Data Class` is also the name of a code smell: a coding pattern that may be a symptom (dấu hiệu) of poor object-oriented design.

## Overview of Data Class Builders

In [2]:
class Coordinate:

    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

moscow = Coordinate(55.76, 37.62)
location = Coordinate(55.76, 37.62)

# __repr__ inherited from object is not very helpful
print(moscow)
print(location)

In [3]:
# the __eq__ method inherited from object compares object IDs
location == moscow

False

In [4]:
(location.lat, location.lon) == (moscow.lat, moscow.lon)

True

Các methods mặc định (__init__, __repr__, and __eq__) ở trên ko được hữu dụng cho lắm.

Here is a Coordinate class built with namedtuple — a factory function that builds a subclass of tuple with the name and fields you specify:

In [5]:
# Example 5-1: namedtuple

from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [6]:
# Useful __repr__
moscow = Coordinate(55.756, 37.617)
moscow

Coordinate(lat=55.756, lon=37.617)

In [7]:
# Meaningful __eq__
moscow == Coordinate(lat=55.756, lon=37.617)

True

The newer `typing.NamedTuple` provides the same functionality, adding a type annotation to each field:

In [9]:
import typing

Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [10]:
# Example 5-2. typing_namedtuple/coordinates.py

from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

In [13]:
# Although NamedTuple appears in the class statement as a superclass,
# it’s actually not. typing.NamedTuple uses the advanced functionality of a metaclass2 to customize the creation of the user’s class.

issubclass(Coordinate, typing.NamedTuple)

TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union

In [12]:
issubclass(Coordinate, tuple)

True

In [ ]:
# Example 5-3. dataclass/coordinates.py

from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

The @dataclass decorator does not depend on inheritance or a metaclass, so it should not interfere with your own use of these mechanisms.

The Coordinate class in Example 5-3 is a subclass of `object`.

### The different data class builders

![img.png](Table5-1.png)

NOTE:

The classes built by typing.NamedTuple and @dataclass have an `__annotations__` attribute holding the type hints for the fields.
However, reading from `__annotations__` directly is not recommended.

(That’s because those functions provide extra services, like resolving forward references in type hints)

Instead, the recommended best practice to get that information is to call
- `inspect.get_annotations(MyClass)` (added in Python 3.10)
- `typing.get_type_hints(MyClass)` (Python 3.5 to 3.9)

Let’s discuss those main features.

#### Mutable instances

- `collections.namedtuple` and `typing.NamedTuple` build tuple subclasses, therefore the instances are immutable.
- By default, `@dataclass` produces mutable classes. When `frozen=True` --> raise exception khi mình đổi giá trị

## 1. Classic Named Tuples
`collections.namedtuple` : a factory that builds subclasses of `tuple` enhanced with field names, a class name, and an informative `__repr__`

Dùng khi cần `tuples` (Thực tế thì Python standard return nhiều tuples)

In [30]:
from collections import namedtuple

# Class: City , Field name: ...
City = namedtuple('City', 'name country population coordinates, reference', defaults=['WGS84'])

tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
print(tokyo)

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667), reference='WGS84')


In [15]:
tokyo.coordinates

(35.689722, 139.691667)

In [20]:
tokyo[3][0]

35.689722

In [21]:
City._fields

('name', 'country', 'population', 'coordinates')

In [22]:
Coordinate = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))

# ._make() builds City from an iterable; City(*delhi_data) would do the same.
delhi = City._make(delhi_data)

# ._asdict() returns a dict built from the named tuple instance.
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

In [24]:
import json
json.dumps(delhi._asdict())

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

## 2. Typed Named Tuples
`typing.NamedTuple`

The main feature of `typing.NamedTuple` are the type annotations

In [33]:
from typing import NamedTuple

class Coordinate1(NamedTuple):
    lat: float
    lon: float
    reference: str = 'WGS84'

tokyo = Coordinate1(28.613889, 77.208889)
tokyo

Coordinate1(lat=28.613889, lon=77.208889, reference='WGS84')

## Type Hints 101
Type hints—a.k.a. type annotations

### No Runtime Effect
Think about Python type hints as “documentation that can be verified by IDEs and type checkers.”

In [34]:
# Example 5-9. Python does not enforce type hints at runtime

import typing
class Coordinate(typing.NamedTuple):
    lat: float
    lon: float

trash = Coordinate('Ni!', None)
print(trash)

Coordinate(lat='Ni!', lon=None)


The type hints are intended primarily to support third-party type checkers, like `Mypy` or the `PyCharm IDE` built-in type checker.

### The Meaning of Variable Annotations

We saw in “No Runtime Effect” that type hints have no effect at runtime.
But at *import time* — when a module is loaded—Python does read them to build the `__annotations__` dictionary that typing.`NamedTuple` and `@dataclass` then use to enhance the class.

In [35]:
class DemoPlainClass:
    a: int
    b: float = 1.1
    c = 'spam'

DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [39]:
# The `a` survives only as an annotation. It doesn’t become a class attribute because no value is bound to it

DemoPlainClass.a

AttributeError: type object 'DemoPlainClass' has no attribute 'a'

In [37]:
DemoPlainClass.b

1.1

In [38]:
DemoPlainClass.c

'spam'

In [40]:
# Example 5-11. meaning/demo_nt.py: a class built with typing.NamedTuple

import typing

class DemoNTClass(typing.NamedTuple):
    a: int
    b: float = 1.1
    c = 'spam'

DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [41]:
# OK
DemoNTClass.a

_tuplegetter(0, 'Alias for field number 0')

In [42]:
DemoNTClass.__doc__

'DemoNTClass(a, b)'

## 3. About @dataclass

```
@dataclass(*, init=True, repr=True, eq=True, order=False,
              unsafe_hash=False, frozen=False)
```

![img.png](Table5-2.png)

### Field Options

In [44]:
# Example 5-13. dataclass/club_wrong.py: this class raises ValueError
from dataclasses import dataclass

@dataclass
class ClubMember:
    name: str
    guests: list = []

ValueError: mutable default <class 'list'> for field guests is not allowed: use default_factory

In [48]:
# Example 5-14. dataclass/club.py: this ClubMember definition works

from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list) # instead of a literal list, the default value is set by calling the `dataclasses.field` function with `default_factory=list`

# Can code: guests: list[str] =... in Python 3.9

![img.png](Table5-3.png)

### Post-init Processing

When that method exists, `@dataclass` will add code to the generated `__init__` to call `__post_init__` as the last step.

### Initialization Variables That Are Not Fields
`InitVar` will prevent `@dataclass` from treating database as a regular field

In [54]:
from dataclasses import dataclass, InitVar

@dataclass
class C:
    i: int
    j: int = None
    database: InitVar[str] = None # init-only variables

    def __post_init__(self, database):
        if self.j is None and database is not None:
            self.j = database.find('j')

c = C(10, database='Huy')

### @dataclass Example: Dublin Core Resource Record


The Dublin Core Schema is a small set of vocabulary terms that can be used to describe digital resources (video, images, web pages, etc.), as well as physical resources such as books or CDs, and objects like artworks


In [1]:
from dataclasses import dataclass, field, fields
from typing import Optional
from enum import Enum, auto
from datetime import date


# Enum will provide type-safe values for the Resource.type field.
class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()

@dataclass
class Resource:
    """Media resource description"""
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK # type field default is ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)

    # The __repr__ generated by @dataclass is OK, but we can make it more readable.
    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']
        for f in fields(cls):
            value = getattr(self, f.name)
            res.append(f'{indent}{f.name} = {value!r},')
        res.append(')')
        return '\n'.join(res)

description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition',
                ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19),
                ResourceType.BOOK, description, 'EN',
                ['computer programming', 'OOP'])

book  # doctest: +NORMALIZE_WHITESPACE


#Without modify __repr__

#Resource(identifier='978-0-13-475759-9', title='Refactoring, 2nd Edition', creators=['Martin Fowler', 'Kent Beck'], date=datetime.date(2018, 11, 19), type=<ResourceType.BOOK: 1>, description='Improving the design of existing code', language='EN', subjects=['computer programming', 'OOP'])

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

## Data Class as a Code Smell

In Refactoring: Improving the Design of Existing Code, 2nd ed. (Addison-Wesley), Martin Fowler and Kent Beck present a catalog of “code smells”—patterns in code that may indicate the need for refactoring. The entry titled “Data Class” starts like this:

    - These are classes that have only fields, getting and setting methods for fields.
    Such classes are dumb data holders and are often being manipulated in far too much detail by other classes.

"Smell of the week" - hàng tuần kiếm ra 1 smell của code và trình bày.

The main idea of object-oriented programming is to place behavior and data together in the same code unit: a class. Nếu class được dùng nhiều, mà có ít/ko có behaviors nào --> behaviors nằm rải rác/duplicate ở đâu đó

### Data Class as Scaffolding (giàn giáo)

Tại tình huống này, Data class là khởi tạp, đơn giản để bắt đầu ---> Sau đó, class sẽ có methods riêng, thay vì dựa vào các class khác

Giàn giáo là tạm thời ---> Cuối cùng class hoàn thiện

### Data Class as Intermediate Representation

In this scenario, the data class instances should be handled as immutable objects—even if the fields are mutable, you should not change them while they are in this intermediate form. If you do, you’re losing the key benefit of having data and behavior close together. When importing/exporting requires changing values, you should implement your own builder methods instead of using the given “as dict” methods or standard constructors.


## Pattern Matching Class Instances

Class patterns are designed to match class instances by type and—optionally—by attributes.

There are three variations of class patterns: simple, keyword, and positional.


### Simple Class Patterns

```
case [str(name), _, _, (float(lat), float(lon))]:
```
That pattern matches a 4-item sequence where:
  - the first item must be an instance of str
  - the last item must be a 2-tuple with two instances of float.

```
    match x:
        case float:  # DANGER!!! --> Matches any subject, because Python sees float as a variable
            do_something_with(x)
```

The simple pattern syntax of `float(x)` is a special case that applies only to nine blessed built-in types

```
bytes   dict   float   frozenset   int   list   set   str   tuple
```

### Keyword Class Patterns
Consider the following City class and five instances
- De doc, nhung kha dai dong

In [1]:
import typing

class City(typing.NamedTuple):
    continent: str
    name: str
    country: str


cities = [
    City('Asia', 'Tokyo', 'JP'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('South America', 'São Paulo', 'BR'),
]

In [2]:
def match_asian_cities():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia'):
                results.append(city)
    return results

match_asian_cities()

[City(continent='Asia', name='Tokyo', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [3]:
def match_asian_countries():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia', country=cc):
                results.append(cc)
    return results

match_asian_countries()

['JP', 'IN']

### Positional Class Patterns
more convenient, but they require explicit (ro rang) support by the class of the subject

In [4]:
def match_asian_cities_pos():
    results = []
    for city in cities:
        match city:
            case City('Asia'): # first attribute value is 'Asia'
                results.append(city)
    return results

match_asian_cities_pos()

[City(continent='Asia', name='Tokyo', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [6]:
def match_asian_countries_pos():
    results = []
    for city in cities:
        match city:
            case City('Asia', _, country): # country variable is bound to the third attribute of the instance
                results.append(country)
    return results

match_asian_countries_pos()

['JP', 'IN']

In [7]:
# Show thu tu cac bien, ham nay tu dong create khi class dc khoi tao
City.__match_args__

('continent', 'name', 'country')

# Chapter Summary

The main topic of this chapter was the data class builders
- `collections.namedtuple`,
- `typing.NamedTuple`,
- `dataclasses.dataclass`

 In particular, both named `tuple` variants produce tuple subclasses, adding only the ability to access fields by name, and providing a `_fields` class attribute listing the field names as a tuple of strings.

Next we studied the main features of the three class builders side by side:
- how to extract instance data as a `dict`
- how to get the names and default values of fields
- how to make a new instance from an existing one

This prompted our first look into *type hints* . (đặc biệt dùng cho chú thích kiểu của thuộc tính) VD: Mypy-->   no effect at all at runtime. Python remains a dynamic language.
Look the syntax from PEP 526 + the effect of annotations in a plain class and in classes built by `typing.NamedTuple` and `@dataclass`.

Next, we covered the most commonly used features provided by `@dataclass` and the `default_factory` option of the `dataclasses.field` function. We also looked into the special pseudotype hints `typing.ClassVar` and `dataclasses.InitVar` that are important in the context of data classes. This main topic concluded with an example based on the Dublin Core Schema, which illustrated how to use `dataclasses.fields` to iterate over the attributes of a `Resource` instance in a custom `__repr__`.

Then, we warned against possible abuse of data classes defeating a basic principle of object-oriented programming: data and the functions that touch it should be together in the same class.

In the last section, we saw how pattern matching works.